**Wikipedia model to "learn" spanish** 

In [ ]:
import zipfile
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/wiki_art.zip"

wikipedia_articles/'Abd al-Salām al-'Uŷaylī.txt:  mismatching "local" filename (wikipedia_articles/'Abd al-Sal─Бm al-'U┼╖ayl─л.txt),
         continuing with "central" filename version
wikipedia_articles/1 − 2 + 3 − 4 + · · ·.txt:  mismatching "local" filename (wikipedia_articles/1 тИТ 2 + 3 тИТ 4 + ┬╖ ┬╖ ┬╖.txt),
         continuing with "central" filename version
wikipedia_articles/Adrian Păunescu.txt:  mismatching "local" filename (wikipedia_articles/Adrian P─Гunescu.txt),
         continuing with "central" filename version
wikipedia_articles/Aeropuerto de Bratislava-Milan Rastislav Štefánik.txt:  mismatching "local" filename (wikipedia_articles/Aeropuerto de Bratislava-Milan Rastislav ┼аtef├бnik.txt),
         continuing with "central" filename version
wikipedia_articles/Affection (álbum de Kumi Kōda).txt:  mismatching "local" filename (wikipedia_articles/Affection (├бlbum de Kumi K┼Нda).txt),
         continuing with "central" filename version
wikipedia_articles/Agapē.txt:  mismat

In [8]:
path_art="/content/wikipedia_articles"